# Smart Stock Prediction #
John, a hacker has got access to database of one of Leading and most profitable trading firm of India. Database had daily share prices, buy date and sell date data from Jan, 2000 till Nov,2019. 
He also found expected share prices for next 2 years from another table. For earning profits using this data, he started searching on web and encountered many unknown terms such as RNN, Attention Models etc. Then he remembered, he has few friends who are working on Deep learning and asked for their help. Can you help him by creating a deep learning based model for predicting buy and sell date.

In [130]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sharepricedateprediction/Smart Stock Prediction.docx
/kaggle/input/sharepricedateprediction/share_train_data.csv
/kaggle/input/sharepricedateprediction/share_test_data.csv


## Data Visulization ##

Training Data (share_train_data.csv)
1. Each row represent one-month data for one particular share. 
2. Share is only bought once in that month and sold in same month only. Buy date will be always before sell date.
3. In below example, share_4 was bought on 8, Jan, 2000 and sold on 10, Jan,2000.

	Share_Name, Year, Month, Share Prices (28-31 values depending on month), Buy Date, Sell Date

	'share_4', 2000, 'JAN', 46.6, 54.7, 66.7, 44.7, 46.0, 72.8, 73.6, 37.9, 49.7, 74.3, 71.2, 51.0, 40.3, 57.8, 70.5, 66.2, 69.8, 45.8, 57.4, 50.2, 62.6, 40.6, 48.7, 71.2, 51.4, 61.4, 49.2, 52.7, 54.9, 37.3, 47.2, 8, 10


In [131]:
train_data = pd.read_csv('/kaggle/input/sharepricedateprediction/share_train_data.csv', header=None)
print(len(train_data))
train_data.head(10)

12189


,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,share_1,2000,JAN,345.0,376.4,344.4,361.7,347.5,350.2,383.8,...,358.6,383.5,366.9,373.7,367.8,347.0,379.0,352.8,3.0,7.0
1,share_2,2000,JAN,114.6,87.2,114.3,85.6,107.9,94.3,81.6,...,116.1,102.0,94.9,93.4,98.7,112.6,92.5,113.6,7.0,24.0
2,share_3,2000,JAN,286.3,271.4,272.9,298.4,265.1,266.3,274.4,...,272.6,270.5,281.0,276.3,287.8,289.2,300.8,293.7,15.0,30.0
3,share_4,2000,JAN,278.9,263.3,256.2,287.8,280.0,267.2,272.5,...,260.0,284.1,281.0,274.4,274.9,272.5,269.3,255.2,13.0,18.0
4,share_5,2000,JAN,154.4,163.1,136.9,138.9,145.5,158.3,165.5,...,167.0,137.1,163.1,135.6,158.5,148.0,136.2,162.8,23.0,24.0
5,share_6,2000,JAN,338.5,321.0,310.4,332.8,324.8,299.6,313.1,...,338.8,303.4,326.7,306.1,301.7,301.7,339.2,323.1,6.0,30.0
6,share_7,2000,JAN,200.2,224.2,232.6,203.7,214.8,206.3,219.8,...,231.3,194.2,217.4,200.0,227.3,205.8,213.8,200.7,10.0,24.0
7,share_8,2000,JAN,401.1,415.2,408.6,399.8,383.8,396.8,398.8,...,413.8,411.9,386.0,413.2,407.9,396.1,406.8,410.1,10.0,22.0
8,share_9,2000,JAN,76.8,57.6,74.9,55.6,77.0,75.6,60.3,...,85.2,86.1,73.1,67.4,66.9,63.9,49.5,64.9,22.0,25.0
9,share_10,2000,JAN,435.2,452.2,447.7,437.3,424.4,454.9,439.5,...,431.1,422.7,454.4,431.9,423.5,434.0,443.3,442.0,16.0,26.0


In [132]:
train_data[40:60]

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
40,share_41,2000,JAN,149.1,137.4,153.7,129.8,128.1,144.0,144.0,...,136.6,145.7,127.1,156.1,149.0,146.7,152.3,152.6,12.0,14.0
41,share_42,2000,JAN,99.4,87.0,92.8,103.1,120.9,86.7,107.1,...,90.5,89.4,108.2,114.0,122.2,96.1,88.8,111.8,6.0,28.0
42,share_43,2000,JAN,460.1,459.0,438.9,468.4,459.8,440.2,452.3,...,464.2,438.1,450.1,463.3,460.0,476.4,440.4,464.8,25.0,29.0
43,share_44,2000,JAN,324.3,314.0,320.7,336.7,312.0,337.8,315.4,...,308.0,317.9,327.4,308.4,311.3,334.6,310.6,320.9,10.0,15.0
44,share_45,2000,JAN,142.6,128.6,150.8,142.5,137.0,161.6,159.4,...,161.9,164.0,137.1,156.2,145.3,152.8,132.3,151.6,2.0,11.0
45,share_46,2000,JAN,450.9,430.5,430.2,428.5,435.6,432.0,451.3,...,459.7,447.8,421.8,442.7,434.8,427.1,445.6,453.8,22.0,24.0
46,share_47,2000,JAN,500.5,475.3,491.6,501.7,500.6,495.1,498.3,...,470.4,471.0,486.9,498.3,493.5,482.4,472.1,483.7,16.0,17.0
47,share_48,2000,JAN,257.2,231.9,265.1,261.5,255.9,251.7,231.7,...,256.0,243.3,256.4,242.7,228.7,242.0,244.2,253.2,9.0,22.0
48,share_49,2000,JAN,247.7,214.1,235.9,243.3,235.4,228.3,217.4,...,220.6,229.5,216.3,217.1,248.0,239.7,225.8,240.7,2.0,28.0
49,share_50,2000,JAN,168.3,149.0,184.3,180.7,183.8,175.0,171.1,...,170.7,149.3,164.2,150.2,161.1,184.7,146.3,165.3,22.0,29.0


In [133]:
# for the first column we have different share classes lets check all the available values
print(len(train_data[0].unique()))
train_data[0].unique()

51


array(['share_1', 'share_2', 'share_3', 'share_4', 'share_5', 'share_6',
       'share_7', 'share_8', 'share_9', 'share_10', 'share_11',
       'share_12', 'share_13', 'share_14', 'share_15', 'share_16',
       'share_17', 'share_18', 'share_19', 'share_20', 'share_21',
       'share_22', 'share_23', 'share_24', 'share_25', 'share_26',
       'share_27', 'share_28', 'share_29', 'share_30', 'share_31',
       'share_32', 'share_33', 'share_34', 'share_35', 'share_36',
       'share_37', 'share_38', 'share_39', 'share_40', 'share_41',
       'share_42', 'share_43', 'share_44', 'share_45', 'share_46',
       'share_47', 'share_48', 'share_49', 'share_50', 'share_51'],
      dtype=object)

In [134]:
# for the second column we have different share classes lets check all the available values
print(len(train_data[1].unique()))
train_data[1].unique()

20


array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [135]:
# for the third column we have different share classes lets check all the available values
print(len(train_data[2].unique()))
train_data[2].unique()

12


array(['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC'], dtype=object)

## Preprocessing ##

In [136]:
#extracting last two columns from Traning data

month_dict={
    'JAN':1, 'FEB':2, 'MAR':3, 'APR':4, 'MAY':5, 'JUN':6,'JUL':7, 'AUG':8,'SEP':9, 'OCT':10, 'NOV':11, 'DEC':12
}

from calendar import monthrange


def dateExtractor(train_data):
    buy_dates=[]
    sell_dates=[]
    
    for row in train_data.values:
        days_in_month = monthrange(int(row[1]), int(month_dict[row[2]]))[1]
        sell_index=days_in_month+4
        buy_index=days_in_month+3
        buy_dates.append(row[buy_index])
        sell_dates.append(row[sell_index])
        
    return buy_dates, sell_dates
                               
buy_dates, sell_dates = dateExtractor(train_data)

In [137]:
print('Unique Buy Dates: '+str(set(buy_dates))+'\nUnique Sell Date: '+str(set(sell_dates)))

Unique Buy Dates: {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0}
Unique Sell Date: {2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0}


In [138]:
# Fill all the Nan value with 0
train_data=train_data.fillna(0)
train_data[99:100]

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
99,share_49,2000,FEB,215.1,217.0,216.2,247.7,235.1,221.4,251.3,...,238.4,232.5,236.5,229.6,252.3,227.8,1.0,20.0,0.0,0.0


Our Dataset don't have same length of share price for all the records because of different days in each month, RNN will expect to have same size of records so we need to preprocess to generate consistence records.

In [141]:
def targetDateReplacer(train_data):
    '''
    Replace buy date and sell date with zeros so that we can generate consitent data 
    '''
    df = pd.DataFrame(columns = [0,1,2,3,4,5,6,7,8,9,10,11,12,13, 14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35])
    for index,row in enumerate(train_data.values):
        days_in_month = monthrange(int(row[1]), int(month_dict[row[2]]))[1]
        sell_index = days_in_month+4
        buy_index = days_in_month+3
        row[sell_index] = 0
        row[buy_index] = 0
        df.loc[index]=row
    return df

modified_df = targetDateReplacer(train_data)
modified_df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,share_1,2000,JAN,345.0,376.4,344.4,361.7,347.5,350.2,383.8,...,358.6,383.5,366.9,373.7,367.8,347.0,379.0,352.8,0,0
1,share_2,2000,JAN,114.6,87.2,114.3,85.6,107.9,94.3,81.6,...,116.1,102.0,94.9,93.4,98.7,112.6,92.5,113.6,0,0
2,share_3,2000,JAN,286.3,271.4,272.9,298.4,265.1,266.3,274.4,...,272.6,270.5,281.0,276.3,287.8,289.2,300.8,293.7,0,0
3,share_4,2000,JAN,278.9,263.3,256.2,287.8,280.0,267.2,272.5,...,260.0,284.1,281.0,274.4,274.9,272.5,269.3,255.2,0,0
4,share_5,2000,JAN,154.4,163.1,136.9,138.9,145.5,158.3,165.5,...,167.0,137.1,163.1,135.6,158.5,148.0,136.2,162.8,0,0


In [142]:
modified_df=modified_df.drop([34,35], axis=1)

In [143]:
#we need to one hot encode sharetype, year, and month column

from sklearn.preprocessing import LabelEncoder
def label_encoder(modified_df):
    cols = (0, 1, 2)
    # process columns, apply LabelEncoder to categorical features
    for c in cols:
        lbl = LabelEncoder() 
        lbl.fit(list(modified_df[c].values)) 
        modified_df[c] = lbl.transform(list(modified_df[c].values))
    
    return modified_df

one_hot_df=label_encoder(modified_df)
one_hot_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0,0,4,345.0,376.4,344.4,361.7,347.5,350.2,383.8,...,354.4,365.2,358.6,383.5,366.9,373.7,367.8,347.0,379.0,352.8
1,11,0,4,114.6,87.2,114.3,85.6,107.9,94.3,81.6,...,98.5,100.7,116.1,102.0,94.9,93.4,98.7,112.6,92.5,113.6
2,22,0,4,286.3,271.4,272.9,298.4,265.1,266.3,274.4,...,276.5,292.8,272.6,270.5,281.0,276.3,287.8,289.2,300.8,293.7
3,33,0,4,278.9,263.3,256.2,287.8,280.0,267.2,272.5,...,287.3,267.7,260.0,284.1,281.0,274.4,274.9,272.5,269.3,255.2
4,44,0,4,154.4,163.1,136.9,138.9,145.5,158.3,165.5,...,145.2,131.6,167.0,137.1,163.1,135.6,158.5,148.0,136.2,162.8


In [144]:
# one hot encoded value of first column
modified_df[0].unique()

array([ 0, 11, 22, 33, 44, 47, 48, 49, 50,  1,  2,  3,  4,  5,  6,  7,  8,
        9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27,
       28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46])

In [145]:
# one hot encoded value of second column
modified_df[1].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [146]:
# one hot encoded value of third column
modified_df[2].unique()

array([ 4,  3,  7,  0,  8,  6,  5,  1, 11, 10,  9,  2])

### Normalization ###

In [147]:
from sklearn import preprocessing
def normalize_data(df):
    min_max_scaler = preprocessing.MinMaxScaler()
    for i in range(34):
        df[i] = min_max_scaler.fit_transform(df[i].values.reshape(-1,1))
    return df

norm_df=normalize_data(modified_df)
norm_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.00,0.0,0.363636,0.657934,0.724446,0.656237,0.692848,0.662974,0.669010,0.739945,...,0.677378,0.700298,0.686249,0.739251,0.704279,0.718451,0.706371,0.685906,0.749308,0.697509
1,0.22,0.0,0.363636,0.167199,0.108603,0.166454,0.105151,0.152535,0.123962,0.096829,...,0.132794,0.137292,0.170072,0.140060,0.125186,0.121941,0.132964,0.222574,0.182879,0.224595
2,0.44,0.0,0.363636,0.532907,0.500852,0.504044,0.558110,0.487431,0.490309,0.507129,...,0.511598,0.546190,0.503193,0.498723,0.521397,0.511173,0.535905,0.571654,0.594701,0.580664
3,0.66,0.0,0.363636,0.517146,0.483603,0.468497,0.535547,0.519173,0.492226,0.503086,...,0.534582,0.492763,0.476373,0.527671,0.521397,0.507129,0.508417,0.538644,0.532424,0.504547
4,0.88,0.0,0.363636,0.251970,0.270230,0.214559,0.218604,0.232637,0.260277,0.275378,...,0.232177,0.203065,0.278416,0.214772,0.270385,0.211747,0.260388,0.292548,0.269276,0.321866


In [148]:
norm_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
count,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,...,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000,12189.000000
mean,0.500000,0.497908,0.501331,0.529440,0.528849,0.528717,0.528758,0.529021,0.528687,0.528599,...,0.528371,0.528692,0.528827,0.528644,0.528736,0.528472,0.528967,0.526890,0.515311,0.327167
std,0.294404,0.302403,0.313815,0.283339,0.283338,0.282869,0.283061,0.283242,0.283435,0.283023,...,0.282884,0.283243,0.282942,0.282993,0.283308,0.283201,0.283437,0.288463,0.296082,0.342258
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.240000,0.210526,0.272727,0.256230,0.256388,0.257344,0.257344,0.255006,0.256443,0.255799,...,0.256012,0.256279,0.254789,0.256492,0.253779,0.256438,0.255913,0.279304,0.266904,0.000000
50%,0.500000,0.473684,0.545455,0.537167,0.538118,0.538527,0.537463,0.538347,0.538019,0.539264,...,0.536072,0.537037,0.537250,0.537037,0.539067,0.538412,0.539314,0.545365,0.535192,0.230526
75%,0.760000,0.736842,0.818182,0.768690,0.767249,0.767774,0.768412,0.770345,0.767625,0.766972,...,0.766120,0.769051,0.766709,0.766922,0.766021,0.767397,0.766461,0.770310,0.767102,0.650257
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [149]:
# we also need to normalize target values
min_max_scaler = preprocessing.MinMaxScaler()
n_buy_dates = min_max_scaler.fit_transform(np.asarray(buy_dates).reshape(-1,1))
n_sell_dates = min_max_scaler.fit_transform(np.asarray(sell_dates).reshape(-1,1))

# lets create target value using list comprehension 
n_buy_dates=list(n_buy_dates)
n_sell_dates=list(n_sell_dates)
target=[[n_buy_dates[i].item(),n_sell_dates[i].item()] for i in range(len(n_sell_dates))]
target[:10]

[[0.06896551724137931, 0.17241379310344826],
 [0.2068965517241379, 0.7586206896551724],
 [0.4827586206896552, 0.9655172413793104],
 [0.41379310344827586, 0.5517241379310345],
 [0.7586206896551725, 0.7586206896551724],
 [0.1724137931034483, 0.9655172413793104],
 [0.3103448275862069, 0.7586206896551724],
 [0.3103448275862069, 0.689655172413793],
 [0.7241379310344828, 0.793103448275862],
 [0.5172413793103449, 0.8275862068965517]]

In [198]:
#split_dataset into 80% training , 10% test and 10% Validation Dataset
train_x=np.array(norm_df[:int(0.90*len(norm_df))])
train_y=np.array(target[:int(0.90*len(target))])
valid_x=np.array(norm_df[int(0.90*len(norm_df)):])
valid_y=np.array(target[int(0.90*len(target)):])
print(len(train_y), len(train_x), len(valid_y), len(valid_x))

10970 10970 1219 1219


In [199]:
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1],1))
valid_x = np.reshape(valid_x, (valid_x.shape[0], valid_x.shape[1],1))

In [200]:
train_x.shape

(10970, 34, 1)

In [226]:
n_features=34
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

regressor = Sequential()
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
regressor.add(Dense(100))
regressor.add(Dense(2))
regressor.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 34, 50)            10400     
_________________________________________________________________
dropout_21 (Dropout)         (None, 34, 50)            0         
_________________________________________________________________
lstm_33 (LSTM)               (None, 34, 50)            20200     
_________________________________________________________________
dropout_22 (Dropout)         (None, 34, 50)            0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 34, 50)            20200     
_________________________________________________________________
dropout_23 (Dropout)         (None, 34, 50)            0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 50)              

In [227]:
from keras import optimizers

# All parameter gradients will be clipped to
# a maximum norm of 1.
sgd = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
regressor.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['mae', 'acc'])

In [ ]:
regressor.fit(train_x, train_y, epochs=5, batch_size=16)

Epoch 1/5
10970/10970 [==============================] - 52s 5ms/step - loss: 0.1216 - mae: 0.2815 - acc: 0.9105
Epoch 2/5
10970/10970 [==============================] - 50s 5ms/step - loss: 0.0656 - mae: 0.2153 - acc: 0.9142
Epoch 3/5
10970/10970 [==============================] - 51s 5ms/step - loss: 0.0652 - mae: 0.2141 - acc: 0.9143
Epoch 4/5
10352/10970 [===========================>..] - ETA: 2s - loss: 0.0656 - mae: 0.2152 - acc: 0.9138

In [215]:
prediction=regressor.predict(valid_x[:32])

In [216]:
print(prediction[:5])
print(valid_y[:5])

[[0.35263115 0.62664974]
 [0.3526469  0.6261008 ]
 [0.3526392  0.6269698 ]
 [0.35282797 0.6254698 ]
 [0.35264826 0.62712896]]
[[0.48275862 0.75862069]
 [0.10344828 0.72413793]
 [0.89655172 1.        ]
 [0.20689655 0.55172414]
 [0.13793103 0.37931034]]


Testing Data (share_test_data.csv)
1. Predict buy and sell date for estimated shares prices for next 2 years.
2. You can only buy once in month and sell once in month for given share.
3. Testing data have same format as training data except buy and sell date are not present.

In [211]:
test_data = pd.read_csv('/kaggle/input/sharepricedateprediction/share_test_data.csv', header=None)
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,share_1,2019,DEC,354.1,344.4,346.9,367.2,374.6,372.7,377.7,...,351.9,379.1,375.2,359.7,350.9,361.7,380.4,360.7,357.0,359.3
1,share_2,2019,DEC,89.5,84.5,84.4,81.2,78.1,87.3,94.3,...,104.1,107.9,89.4,88.2,79.7,83.4,96.7,114.9,96.8,108.2
2,share_3,2019,DEC,266.8,278.3,265.3,280.3,301.2,271.6,288.7,...,278.7,263.0,290.6,271.2,302.2,294.9,275.5,281.7,293.9,268.8
3,share_4,2019,DEC,285.6,277.1,271.3,259.9,283.2,264.7,254.6,...,263.4,285.0,257.4,272.5,258.0,255.8,269.5,289.1,269.6,252.3
4,share_5,2019,DEC,164.6,168.3,149.0,139.0,132.0,167.7,137.4,...,164.2,168.6,159.2,133.9,136.0,140.6,165.3,152.0,170.7,143.0


In [212]:
en_test=label_encoder(test_data)
norm_data=normalize_data(en_test)
norm_data.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.00,0.0,0.181818,0.676288,0.656576,0.661542,0.705030,0.719752,0.716208,0.726827,...,0.673792,0.729126,0.723064,0.688950,0.668886,0.692653,0.731263,0.691848,0.682479,0.686731
1,0.22,0.0,0.181818,0.110541,0.102537,0.101004,0.095482,0.085934,0.106769,0.122949,...,0.144079,0.147027,0.113292,0.110922,0.086536,0.098249,0.123769,0.167307,0.126496,0.144984
2,0.44,0.0,0.181818,0.489630,0.515668,0.487294,0.519821,0.562847,0.500320,0.537183,...,0.517315,0.479931,0.542565,0.500532,0.564312,0.549979,0.506638,0.523261,0.547650,0.491478
3,0.66,0.0,0.181818,0.529827,0.513110,0.500107,0.476343,0.524369,0.485586,0.464522,...,0.484609,0.527152,0.471730,0.503300,0.469401,0.466467,0.493790,0.539052,0.495726,0.455879
4,0.88,0.0,0.181818,0.271114,0.281177,0.238949,0.218670,0.201154,0.278454,0.214788,...,0.272552,0.277313,0.262215,0.208218,0.207430,0.220419,0.270664,0.246479,0.284402,0.220065


Expected Output
1. Python code for training model using given train data using Tensorflow, Pytorch or Keras frameworks.
2. Python code for predicting on test data using trained model.
3. Generated predictions for test data in same format as training data. 
4. A PDF report explaining your approach and results. 